In [1]:
import os
from os.path import exists
import itertools
import json

import pandas as pd
import numpy as np

from os import listdir
from os.path import isfile, join

In [2]:
print(os.getcwd())

/mnt/compbio_tools/NS-23.0061/NS-23.0061_analysis_code


In [3]:
experiment_id = "NS-23.0061"

samples_to_remove_from_metadata = [] # ['K20A941', '52_A94_2']

metadata_subdir = os.path.expanduser("~/fht.samba.data/experiments/ATACseq/{}/analysis/metadata/".format(experiment_id))
print('metadata_subdir exists:{}\n'.format(os.path.exists(metadata_subdir)))

metadata_subdir exists:True



In [4]:
def read_metadata(metadata_subdir, samples_to_remove_from_metadata):
    
    metadata_files = [f for f in listdir(metadata_subdir) if isfile(join(metadata_subdir, f))]
    print('Files in metadata subdirectory:\n{}\n'.format(metadata_files))
    input_metadata_dir = list(filter(lambda x: ('_metadata_r' in x and experiment_id in x), metadata_files))[0]
    print('{} exists:{}\n'.format(input_metadata_dir, exists(metadata_subdir + input_metadata_dir)))
    
    metadata_df = pd.read_csv(
       metadata_subdir + input_metadata_dir,
        sep = "\t",
        index_col = 0)
    metadata_df.drop(samples_to_remove_from_metadata, inplace=True)
    print("metadata_df.shape:  {}".format(metadata_df.shape))
    return metadata_df

metadata_df = read_metadata(metadata_subdir, samples_to_remove_from_metadata)
metadata_df.head()

Files in metadata subdirectory:
['ATACseq_ARID1AKO_ARID1BKD.csv', 'ATACseq_ARID1AKO_ARID1BKD.txt', 'NS-23.0061_metadata_r12x34.txt', 'ATACseq_ARID1AKO_ARID1BKD.xlsx', 'NS-23.0061_group_dict.json', 'NS-23.0061_contrast_dict.json', 'NS-23.0061_contrasts_r5x3.txt']

NS-23.0061_metadata_r12x34.txt exists:True

metadata_df.shape:  (12, 34)


,assay,bio_sample_type,donor_id,in_vivo_animal_id,in_vivo_tissue_type,bio_context_parental_id,bio_context_id,bio_context_modification,special_media_for_bio_context,pert_id,...,sample_prep_ELN_entry_URL,ChIP-seq antibody,comments,pert_dose_str,pert_time_str,group,experiment_id,R_group,stripped_bio_context_id,ModelID
sample_id,,,,,,,,,,,,,,,,,,,,,
SRR5876158,ATAC-seq,cell line (in vitro),NaN,NaN,NaN,HCT116,HCT116,NaN,NaN,WT,...,NaN,NaN,NaN,NaN,NaN,WT_HCT116,NS-23.0061,WT_HCT116,HCT116,ACH-000971
SRR5876159,ATAC-seq,cell line (in vitro),NaN,NaN,NaN,HCT116,HCT116,NaN,NaN,WT,...,NaN,NaN,NaN,NaN,NaN,WT_HCT116,NS-23.0061,WT_HCT116,HCT116,ACH-000971
SRR5876160,ATAC-seq,cell line (in vitro),NaN,NaN,NaN,HCT116,HCT116,NaN,NaN,ARID1BKD,...,NaN,NaN,NaN,NaN,NaN,ARID1BKD_HCT116,NS-23.0061,ARID1BKD_HCT116,HCT116,ACH-000971
SRR5876161,ATAC-seq,cell line (in vitro),NaN,NaN,NaN,HCT116,HCT116,NaN,NaN,ARID1BKD,...,NaN,NaN,NaN,NaN,NaN,ARID1BKD_HCT116,NS-23.0061,ARID1BKD_HCT116,HCT116,ACH-000971
SRR5876162,ATAC-seq,cell line (in vitro),NaN,NaN,NaN,HCT116,HCT116,NaN,NaN,ARID1AKO,...,NaN,NaN,NaN,NaN,NaN,ARID1AKO_HCT116,NS-23.0061,ARID1AKO_HCT116,HCT116,ACH-000971


In [5]:
def create_contrast_group(cur_metadata_df, match_columns):

    # Add the column group to the matching columns
    all_match_columns = match_columns.copy()
    all_match_columns.extend(['group'])

    # Create dataframe with the match_columns and group for DMSO and FHT each 
    negative_control_groups = cur_metadata_df.loc[cur_metadata_df['pert_type'] == 'ctrl_vehicle', all_match_columns].drop_duplicates()
    test_groups = cur_metadata_df.loc[cur_metadata_df['pert_type'] == 'trt_cp', all_match_columns].drop_duplicates()

    my_matched_negative_control_groups = []


    for i in range(len(test_groups)):
        match_locs = list(itertools.repeat(True, len(negative_control_groups)))

        for mc in match_columns:

            a = list(itertools.repeat(test_groups.iloc[i, ][mc], len(negative_control_groups)))
            b = list(negative_control_groups.loc[:, mc])
            match_locs = np.array(match_locs)* np.array([i==j for i, j in zip(a, b)])

        # Ensure that we only have one mathcong DMSO and FHT
        sum_match_locs = np.sum(match_locs)
        assert(1 == sum_match_locs)

        my_matched_negative_control_groups.extend(list(negative_control_groups.group[match_locs]))

    # Create DataFrame
    data = {'test_group': test_groups.group,
            'negative_ctrl_grp': my_matched_negative_control_groups}
    orig_contrasts_df = pd.DataFrame(data).reset_index(drop=True)
    orig_contrasts_df['contrast'] = list(pd.Series(orig_contrasts_df.fillna('').values.tolist()).str.join(' - '))
    orig_contrasts_df['file_friendly_name'] = orig_contrasts_df["contrast"].str.replace(" - ", "_").replace(" ","")
    orig_contrasts_df.set_index('contrast', inplace=True)
    return orig_contrasts_df


match_columns = ["bio_context_id"]
orig_contrasts_df = create_contrast_group(metadata_df, match_columns)
orig_contrasts_df

,test_group,negative_ctrl_grp,file_friendly_name
contrast,,,
ARID1BKD_HCT116 - WT_HCT116,ARID1BKD_HCT116,WT_HCT116,ARID1BKD_HCT116_WT_HCT116
ARID1AKO_HCT116 - WT_HCT116,ARID1AKO_HCT116,WT_HCT116,ARID1AKO_HCT116_WT_HCT116
ARID1AKO_ARID1BKD_HCT116 - WT_HCT116,ARID1AKO_ARID1BKD_HCT116,WT_HCT116,ARID1AKO_ARID1BKD_HCT116_WT_HCT116
ARID1BKD_TOV21G - WT_TOV21G,ARID1BKD_TOV21G,WT_TOV21G,ARID1BKD_TOV21G_WT_TOV21G


In [6]:
### ADDITION FOR THIS EXPERIMENT ###

df = pd.DataFrame(data = { 'test_group' : ['ARID1AKO_ARID1BKD_HCT116', 'ARID1AKO_ARID1BKD_HCT116'],
                          'negative_ctrl_grp' : ['ARID1AKO_HCT116', 'ARID1BKD_HCT116'],
                          'file_friendly_name' : ['ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116', 'ARID1AKO_ARID1BKD_HCT116_ARID1BKD_HCT116']
                         },
    index = ['ARID1AKO_ARID1BKD_HCT116 - ARID1AKO_HCT116', 'ARID1AKO_ARID1BKD_HCT116 - ARID1BKD_HCT116'])
df

orig_contrasts_df = orig_contrasts_df.append(df).iloc[[0,1,2,4,5,3],]
orig_contrasts_df.index.name = 'contrast'
orig_contrasts_df

### ADDITION FOR THIS EXPERIMENT ###

,test_group,negative_ctrl_grp,file_friendly_name
contrast,,,
ARID1BKD_HCT116 - WT_HCT116,ARID1BKD_HCT116,WT_HCT116,ARID1BKD_HCT116_WT_HCT116
ARID1AKO_HCT116 - WT_HCT116,ARID1AKO_HCT116,WT_HCT116,ARID1AKO_HCT116_WT_HCT116
ARID1AKO_ARID1BKD_HCT116 - WT_HCT116,ARID1AKO_ARID1BKD_HCT116,WT_HCT116,ARID1AKO_ARID1BKD_HCT116_WT_HCT116
ARID1AKO_ARID1BKD_HCT116 - ARID1AKO_HCT116,ARID1AKO_ARID1BKD_HCT116,ARID1AKO_HCT116,ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116
ARID1AKO_ARID1BKD_HCT116 - ARID1BKD_HCT116,ARID1AKO_ARID1BKD_HCT116,ARID1BKD_HCT116,ARID1AKO_ARID1BKD_HCT116_ARID1BKD_HCT116
ARID1BKD_TOV21G - WT_TOV21G,ARID1BKD_TOV21G,WT_TOV21G,ARID1BKD_TOV21G_WT_TOV21G


In [7]:
def save_as_txt(experiment_id, orig_contrasts_df, metadata_subdir, name):
       
    # save as csv
    output_filename = experiment_id + name + str(orig_contrasts_df.shape[0]) + 'x' + str(orig_contrasts_df.shape[1]) + '.txt'
    print("output_filename : {}".format(output_filename))

    output_filepath = os.path.join(metadata_subdir, output_filename)
    print("output_filepath : {}".format(output_filepath))

    orig_contrasts_df.to_csv(output_filepath, sep="\t")

    
name = "_contrasts_r"
save_as_txt(experiment_id, orig_contrasts_df, metadata_subdir, name)

output_filename : NS-23.0061_contrasts_r6x3.txt
output_filepath : /home/fuzan/fht.samba.data/experiments/ATACseq/NS-23.0061/analysis/metadata/NS-23.0061_contrasts_r6x3.txt


In [8]:
def mapping_sample_to_group(metadata_df):
    mapping_sample_to_group = {}
    for group in metadata_df.group.unique():
        t = metadata_df.loc[metadata_df.group == group]
        cur_sample_ids = t.index.to_list()
        mapping_sample_to_group[group] = cur_sample_ids
        
    # save as json file
    output_filename = experiment_id + '_group_dict.json'
    print("output_filename : {}".format(output_filename))

    output_filepath = os.path.join(metadata_subdir, output_filename)
    print("output_filepath : {}".format(output_filepath))
    
    with open(output_filepath, "w") as outfile:
        json.dump(mapping_sample_to_group, outfile)
    return mapping_sample_to_group


mapping_sample_to_group = mapping_sample_to_group(metadata_df)
mapping_sample_to_group

output_filename : NS-23.0061_group_dict.json
output_filepath : /home/fuzan/fht.samba.data/experiments/ATACseq/NS-23.0061/analysis/metadata/NS-23.0061_group_dict.json


{'WT_HCT116': ['SRR5876158', 'SRR5876159'],
 'ARID1BKD_HCT116': ['SRR5876160', 'SRR5876161'],
 'ARID1AKO_HCT116': ['SRR5876162', 'SRR5876163'],
 'ARID1AKO_ARID1BKD_HCT116': ['SRR5876164', 'SRR5876165'],
 'WT_TOV21G': ['SRR5876661', 'SRR5876662'],
 'ARID1BKD_TOV21G': ['SRR5876663', 'SRR5876664']}

In [9]:
def mapping_contrast_to_sample(metadata_df, orig_contrasts_df, mapping_sample_to_group):
    
    mapping_contrast_to_sample = orig_contrasts_df.reset_index().set_index('file_friendly_name').transpose().to_dict()
    for contrast in mapping_contrast_to_sample.keys():
        test_group = mapping_contrast_to_sample[contrast]['test_group']
        negative_ctrl_grp = mapping_contrast_to_sample[contrast]['negative_ctrl_grp']

        mapping_contrast_to_sample[contrast]['test_group'] = {test_group: mapping_sample_to_group[test_group]}
        mapping_contrast_to_sample[contrast]['negative_ctrl_grp'] = {negative_ctrl_grp: mapping_sample_to_group[negative_ctrl_grp]}
        
    # save as json file
    output_filename = experiment_id + '_contrast_dict.json'
    print("output_filename : {}".format(output_filename))

    output_filepath = os.path.join(metadata_subdir, output_filename)
    print("output_filepath : {}".format(output_filepath))
    
    with open(output_filepath, "w") as outfile:
        json.dump(mapping_contrast_to_sample, outfile) 
    return mapping_contrast_to_sample
    
mapping_contrast_to_sample = mapping_contrast_to_sample(metadata_df, orig_contrasts_df, mapping_sample_to_group)
mapping_contrast_to_sample

output_filename : NS-23.0061_contrast_dict.json
output_filepath : /home/fuzan/fht.samba.data/experiments/ATACseq/NS-23.0061/analysis/metadata/NS-23.0061_contrast_dict.json


{'ARID1BKD_HCT116_WT_HCT116': {'contrast': 'ARID1BKD_HCT116 - WT_HCT116',
  'test_group': {'ARID1BKD_HCT116': ['SRR5876160', 'SRR5876161']},
  'negative_ctrl_grp': {'WT_HCT116': ['SRR5876158', 'SRR5876159']}},
 'ARID1AKO_HCT116_WT_HCT116': {'contrast': 'ARID1AKO_HCT116 - WT_HCT116',
  'test_group': {'ARID1AKO_HCT116': ['SRR5876162', 'SRR5876163']},
  'negative_ctrl_grp': {'WT_HCT116': ['SRR5876158', 'SRR5876159']}},
 'ARID1AKO_ARID1BKD_HCT116_WT_HCT116': {'contrast': 'ARID1AKO_ARID1BKD_HCT116 - WT_HCT116',
  'test_group': {'ARID1AKO_ARID1BKD_HCT116': ['SRR5876164', 'SRR5876165']},
  'negative_ctrl_grp': {'WT_HCT116': ['SRR5876158', 'SRR5876159']}},
 'ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116': {'contrast': 'ARID1AKO_ARID1BKD_HCT116 - ARID1AKO_HCT116',
  'test_group': {'ARID1AKO_ARID1BKD_HCT116': ['SRR5876164', 'SRR5876165']},
  'negative_ctrl_grp': {'ARID1AKO_HCT116': ['SRR5876162', 'SRR5876163']}},
 'ARID1AKO_ARID1BKD_HCT116_ARID1BKD_HCT116': {'contrast': 'ARID1AKO_ARID1BKD_HCT116 - AR